<a href="https://colab.research.google.com/github/zwfswt/GEE/blob/main/GEE01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Colab和Cloud身份验证

In [1]:
from google.colab import auth
auth.authenticate_user()

2. 验证gee

In [2]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=WjmbWb6zzEimy6bz7qvciARSsdTrqCBvGUjxHN7Egrg&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhoFL6BLChd_36r2ldSV86PxIplHiAGTLLOOLHO0uDnio2Ni75UI7g

Successfully saved authorization token.


3. 导入tf

In [3]:
import tensorflow as tf
print(tf.__version__)

2.8.0


4. 导入folium

In [4]:
import folium
print(folium.__version__)

0.8.3


5. 定义后面需要用到的变量

In [5]:
# gee用户名
USER_NAME = 'shangweitao456'

# 桶名称
OUTPUT_BUCKET = 'swtswt'

# L8表面反射率数据
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# 6个波段
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

# 官网给的98个点的标签
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')
# 标签名称：土地覆盖
LABEL = 'landcover'
# 3个类
N_CLASSES = 3

# 训练数据的名称
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

# 训练-测试前缀-文件格式-存储路径
TRAIN_FILE_PREFIX = 'Training_demo'
TEST_FILE_PREFIX = 'Testing_demo'
file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = 'gs://' + OUTPUT_BUCKET + '/' + TEST_FILE_PREFIX + file_extension

# 预测数据集
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'

# 分类影像输出
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'
# 导出影像范围
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])
# GEE资源名称
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/Classified_pixel_demo'

三个类别（裸露、植被、水）
L8数据
通过使用 folium 进行可视化检查合成

In [6]:
# 云掩膜函数，跟S2的一样
def maskL8sr(image):
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('pixel_qa')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
    qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(BANDS).divide(10000)

# 2018年平均值合成数据，记得好像推荐使用median
image = L8SR.filterDate('2018-01-01', '2018-12-31').map(maskL8sr).median()

# 可视化 getMapId
mapid = image.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3})
# folium.Map
map = folium.Map(location=[38., -122.5])

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map
# 这个地图效果挺好的

标签

In [7]:
# 样本点加载，这个好像从影像上把属性值提取到了
sample = image.sampleRegions(
  collection=LABEL_DATA, properties=[LABEL], scale=30).randomColumn()

# 样本划分为7:3
training = sample.filter(ee.Filter.lt('random', 0.7))
testing = sample.filter(ee.Filter.gte('random', 0.7))

from pprint import pprint

# 打印点的属性
pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

{'training': {'geometry': None,
              'id': '00009f65e3c9ae02b84e_0',
              'properties': {'B2': 0.05220000073313713,
                             'B3': 0.062049999833106995,
                             'B4': 0.03660000115633011,
                             'B5': 0.01140000019222498,
                             'B6': 0.006800000090152025,
                             'B7': 0.005249999929219484,
                             'landcover': 2,
                             'random': 0.4019922015829136},
              'type': 'Feature'}}
{'testing': {'geometry': None,
             'id': '0000642db5938d967908_0',
             'properties': {'B2': 0.05829999968409538,
                            'B3': 0.08560000360012054,
                            'B4': 0.11620000004768372,
                            'B5': 0.24390000104904175,
                            'B6': 0.29600000381469727,
                            'B7': 0.19820000231266022,
                            'landcover

导出训练和测试数据

In [8]:
# 检查桶
print('Found Cloud Storage bucket.' if tf.io.gfile.exists('gs://' + OUTPUT_BUCKET) 
    else 'Can not find output Cloud Storage bucket.')

Found Cloud Storage bucket.


导出数据

In [9]:
# 创建任务
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  fileNamePrefix=TRAIN_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  fileNamePrefix=TEST_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

开始导出任务

In [10]:
# 开始任务
training_task.start()
testing_task.start()

监测任务执行情况

In [11]:
# 打印所有任务
pprint(ee.batch.Task.list())

[<Task 2IA3IBJ6CGONUEOC4MZ2AAWA EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task 4WMOJVTFQCRCUYI7RIFPZFRG EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task V5VI35UGPJSK7JN54BW4LBUZ INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/shangweitao456/Classified_pixel_demo" (FAILED)>,
 <Task GVRJ5EUXRIYYI2YAVNB33XPU EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task T2OTCQUY6GXYTKRDB5JBGMTC EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task 3JCDDKFL4M4LZY4DIUDHT2LK EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task HM76MGHMBAOJH6ONEJYIYSLW EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task KSUHBSS72U2HVXI2BJNT6PA3 EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task U3SD7C3TKG26JCLXFN7CLY4Y EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task NAS53Y4T3M6TJKFCEXZ435M3 EXPORT_FEATURES: ndvi_shp (COMPLETED)>,
 <Task OWM7AOMMMRPAYWI4K5NPINYT EXPORT_IMAGE: Sentinel-NDVI (COMPLETED)>,
 <Task LAI3FYACW754G4NSRRSYG2C3 EXPORT_IMAGE: Sentinel-NDVI (COMPLETED)>,
 <Task T3KDS

检查导出的文件是否存在

In [12]:
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH) 
    else 'No training file found.')
print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH) 
    else 'No testing file found.')

Found training file.
Found testing file.


导出影像

In [13]:
# 斑块大小，compressed：压缩
image_export_options = {
  'patchDimensions': [256, 256],
  'maxFileSize': 104857600,
  'compressed': True
}

# 设置任务
image_task = ee.batch.Export.image.toCloudStorage(
  image=image,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=30,
  fileFormat='TFRecord',
  region=EXPORT_REGION.toGeoJSON()['coordinates'],
  formatOptions=image_export_options,
)

In [14]:
# 开始执行任务
image_task.start()

监控任务进度

In [15]:
# Print all tasks.
pprint(ee.batch.Task.list())

[<Task A4XVDZI3RH4N36IFPZ7WYPMY EXPORT_IMAGE: Image Export (READY)>,
 <Task 2IA3IBJ6CGONUEOC4MZ2AAWA EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task 4WMOJVTFQCRCUYI7RIFPZFRG EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task V5VI35UGPJSK7JN54BW4LBUZ INGEST_IMAGE: Ingest image: "projects/earthengine-legacy/assets/users/shangweitao456/Classified_pixel_demo" (FAILED)>,
 <Task GVRJ5EUXRIYYI2YAVNB33XPU EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task T2OTCQUY6GXYTKRDB5JBGMTC EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task 3JCDDKFL4M4LZY4DIUDHT2LK EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task HM76MGHMBAOJH6ONEJYIYSLW EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task KSUHBSS72U2HVXI2BJNT6PA3 EXPORT_FEATURES: Testing Export (COMPLETED)>,
 <Task U3SD7C3TKG26JCLXFN7CLY4Y EXPORT_FEATURES: Training Export (COMPLETED)>,
 <Task NAS53Y4T3M6TJKFCEXZ435M3 EXPORT_FEATURES: ndvi_shp (COMPLETED)>,
 <Task OWM7AOMMMRPAYWI4K5NPINYT EXPORT_IMAGE: Sentinel-NDVI (COMPLETED)>,
 <Task LAI3FYACW7

In [17]:
# 30s查询一次导出任务是否完成
import time
while image_task.active():
  print('正在执行任务 (id: {}).'.format(image_task.id))
  time.sleep(30)
print('导出完成')

正在执行任务 (id: A4XVDZI3RH4N36IFPZ7WYPMY).
正在执行任务 (id: A4XVDZI3RH4N36IFPZ7WYPMY).
正在执行任务 (id: A4XVDZI3RH4N36IFPZ7WYPMY).
正在执行任务 (id: A4XVDZI3RH4N36IFPZ7WYPMY).
导出完成


数据准备和预处理
将 TFRecord 文件中的数据读取到tf.data.Dataset. 预处理数据集，使其成为适合模型输入的格式。

In [18]:
# 从云存储读取TF文件
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
# 打印数据集
print(iter(train_dataset).next())

tf.Tensor(b'\nw\n\x0e\n\x02B2\x12\x08\x12\x06\n\x04\xab\xcfU=\n\x0e\n\x02B3\x12\x08\x12\x06\n\x04$(~=\n\x0e\n\x02B4\x12\x08\x12\x06\n\x04\xe2\xe9\x15=\n\x0e\n\x02B5\x12\x08\x12\x06\n\x04\x11\xc7:<\n\x0e\n\x02B6\x12\x08\x12\x06\n\x04\x89\xd2\xde;\n\x0e\n\x02B7\x12\x08\x12\x06\n\x041\x08\xac;\n\x15\n\tlandcover\x12\x08\x12\x06\n\x04\x00\x00\x00@', shape=(), dtype=string)


定义数据结构

In [20]:
# 结构
columns = [
  tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES
]

# 字典
features_dict = dict(zip(FEATURE_NAMES, columns))

pprint(features_dict)

{'B2': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B3': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B4': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B5': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B6': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'B7': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None),
 'landcover': FixedLenFeature(shape=[1], dtype=tf.float32, default_value=None)}


解析数据集

In [21]:
def parse_tfrecord(example_proto):
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)


parsed_dataset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)
pprint(iter(parsed_dataset).next())

({'B2': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0522], dtype=float32)>,
  'B3': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.06205], dtype=float32)>,
  'B4': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0366], dtype=float32)>,
  'B5': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0114], dtype=float32)>,
  'B6': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.0068], dtype=float32)>,
  'B7': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.00525], dtype=float32)>},
 <tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>)


创建附加功能

In [22]:
def normalized_difference(a, b):
  nd = (a - b) / (a + b)
  nd_inf = (a - b) / (a + b + 0.000001)
  return tf.where(tf.math.is_finite(nd), nd, nd_inf)

def add_NDVI(features, label):
  features['NDVI'] = normalized_difference(features['B5'], features['B4'])
  return features, label

模型设置
创建Keras模型

In [23]:
from tensorflow import keras

input_dataset = parsed_dataset.map(add_NDVI)

def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())),
          tf.one_hot(indices=label, depth=N_CLASSES))
# 每批次给8个数据
input_dataset = input_dataset.map(to_tuple).batch(8)

# 模型
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(64, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

# 编译模型
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 训练模型
model.fit(x=input_dataset, epochs=10)

Epoch 1/10
9/9 [==============================] - 2s 11ms/step - loss: 1.0711 - accuracy: 0.5139
Epoch 2/10
9/9 [==============================] - 0s 4ms/step - loss: 1.0174 - accuracy: 0.8194
Epoch 3/10
9/9 [==============================] - 0s 10ms/step - loss: 0.9756 - accuracy: 0.9167
Epoch 4/10
9/9 [==============================] - 0s 6ms/step - loss: 0.9376 - accuracy: 0.8889
Epoch 5/10
9/9 [==============================] - 0s 8ms/step - loss: 0.8941 - accuracy: 0.9722
Epoch 6/10
9/9 [==============================] - 0s 8ms/step - loss: 0.8514 - accuracy: 0.9444
Epoch 7/10
9/9 [==============================] - 0s 5ms/step - loss: 0.8316 - accuracy: 0.9444
Epoch 8/10
9/9 [==============================] - 0s 4ms/step - loss: 0.7912 - accuracy: 0.9028
Epoch 9/10
9/9 [==============================] - 0s 8ms/step - loss: 0.7438 - accuracy: 0.9583
Epoch 10/10
9/9 [==============================] - 0s 5ms/step - loss: 0.7289 - accuracy: 0.9583


检查模型准确性

In [24]:
test_dataset = (
  tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')
    .map(parse_tfrecord, num_parallel_calls=5)
    .map(add_NDVI)
    .map(to_tuple)
    .batch(1))

model.evaluate(test_dataset)

26/26 [==============================] - 1s 2ms/step - loss: 0.7068 - accuracy: 1.0000


[0.7067586183547974, 1.0]

使用训练好的模型对GEE图像进行分类
在 Cloud Storage 中查找图像文件和 JSON 混合器文件

In [25]:
# 文件列表
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# 第一个文件
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# 得到第一个影像和JSON混合文件
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

image_files_list.sort()

pprint(image_files_list)
print(json_file)

['gs://swtswt/Image_pixel_demo_00000.tfrecord.gz',
 'gs://swtswt/Image_pixel_demo_00001.tfrecord.gz']
gs://swtswt/Image_pixel_demo_mixer.json


读取JSON文件

In [27]:
# 从json读取影像信息
import json
json_text = !gsutil cat {json_file}
mixer = json.loads(json_text.nlstr)
pprint(mixer)

{'patchDimensions': [256, 256],
 'patchesPerRow': 13,
 'projection': {'affine': {'doubleMatrix': [0.00026949458523585647,
                                            0.0,
                                            -122.70007617412975,
                                            0.0,
                                            -0.00026949458523585647,
                                            38.00089247493765]},
                'crs': 'EPSG:4326'},
 'totalPatches': 130}


把影像读取到数据集

In [28]:
# 从JSON得到相关信息
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS
]

# 解析字典
image_features_dict = dict(zip(BANDS, image_columns))

# 把列表制作成数据集
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')

# 解析函数
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# 把数据解析为张量
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

# 把大张量解析为小张量
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)

# 增加NDVI
image_dataset = image_dataset.map(
  # 增加NDVI，但没有标签
  lambda features: add_NDVI(features, None)[0]
)


image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# 把数据装入批次
image_dataset = image_dataset.batch(patch_width * patch_height)

为每个像素进行预测，输出三个类别的概率

In [29]:
# 预测需要一些时间
predictions = model.predict(image_dataset, steps=patches, verbose=1)

# 打印第一个
print(predictions[0])

130/130 [==============================] - 633s 5s/step
[[0.28799987 0.53123534 0.1807647 ]]


预测结果写入TF文件，类和概率

In [ ]:
print('Writing to file ' + OUTPUT_IMAGE_FILE)

In [ ]:
# 实例化写入函数
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)


patch = [[], [], [], []]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))
  patch[1].append(prediction[0][0])
  patch[2].append(prediction[0][1])
  patch[3].append(prediction[0][2])
  
  if (len(patch[0]) == patch_width * patch_height):
    print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    # 创建一个案例
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'prediction': tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=patch[0])),
          'bareProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[1])),
          'vegProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[2])),
          'waterProb': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=patch[3])),
        }
      )
    )
    
    writer.write(example.SerializeToString())
    patch = [[], [], [], []]
    cur_patch += 1

writer.close()

将分类结果上传到GEE资产
验证预测文件是否存在

In [ ]:
!gsutil ls -l {OUTPUT_IMAGE_FILE}

In [ ]:
print('Uploading to ' + OUTPUT_ASSET_ID)

In [ ]:
# 开始上传
!earthengine upload image --asset_id={OUTPUT_ASSET_ID} --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

检查资产获取状态

In [ ]:
ee.batch.Task.list()

检查上传的资产
将类别概率向量显示为 RGB 图像，其颜色对应于像素中裸露、植被、水的概率。还使用相同的调色板显示获奖班级。

In [ ]:
predictions_image = ee.Image(OUTPUT_ASSET_ID)

prediction_vis = {
  'bands': 'prediction',
  'min': 0,
  'max': 2,
  'palette': ['red', 'green', 'blue']
}
probability_vis = {'bands': ['bareProb', 'vegProb', 'waterProb'], 'max': 0.5}

prediction_map_id = predictions_image.getMapId(prediction_vis)
probability_map_id = predictions_image.getMapId(probability_vis)

map = folium.Map(location=[37.6413, -122.2582])
folium.TileLayer(
  tiles=prediction_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='prediction',
).add_to(map)
folium.TileLayer(
  tiles=probability_map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='probability',
).add_to(map)
map.add_child(folium.LayerControl())
map

这个案例非常好，要仔细研究一下